In [42]:
# !pip install pandas
!pip install fastparquet

     -------------------------------------- 629.0/629.0 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 26.3 MB/s eta 0:00:00
     ---------------------------------------- 143.0/143.0 kB ? eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd

In [44]:
face_points = 468
pose_points = 33
left_hand_points = 21
right_hand_points = 21

# Definicion de Columnas

In [45]:
point_columns = []
for index in range (face_points):
    point_columns.append(f'''face_p{index}_x''')
    point_columns.append(f'''face_p{index}_y''')

for index in range (pose_points):
    point_columns.append(f'''pose_p{index}_x''')
    point_columns.append(f'''pose_p{index}_y''')

for index in range (left_hand_points):
    point_columns.append(f'''left_hand_p{index}_x''')
    point_columns.append(f'''left_hand_p{index}_y''')

for index in range (right_hand_points):
    point_columns.append(f'''right_hand_p{index}_x''')
    point_columns.append(f'''right_hand_p{index}_y''')

point_columns.append("sign")

print("ready")

ready


# Construcción de Dataframe

Al momento de armar el parquet con los datos originales, como estos ocupan mucha memoria en principio, se serparó la lista de señas en 2 conjuntos, con el objetivo de armar 2 parquet que luego son concatenados para armar uno solo.
Los datos originales usados para armar el parquet ocupan 5GB descomprimidos.
Luego con el armado del parquet este alcanza un tamaño de 500MB

In [7]:
def build_parquet(name_to_store):
    data_frame_rows = []
    print(f"Armando parquet: {name_to_store}")
    for sign in range(len(signs_list)):
        for personNumber in range(numberOfPersons):
            for videoNumber in range(numberOfVideosPerPerson):
                results = np.load(os.path.join('./LSA64_data', signs_list[sign], f'''0{str(personNumber)}_0{str(videoNumber)}.npy'''))
                for frame in results:
                  arr = [i for i in frame]
                  arr.append(signs_list[sign])
                  data_frame_rows.append(arr)
    videos_dataframe = pd.DataFrame(data_frame_rows,columns=point_columns)
    videos_dataframe.to_parquet(os.path.join(f'./${name_to_store}'))
    print(f"Parquet almacenado: {name_to_store}")

In [6]:
#DEFINICION DE SEÑAS A UTILIZAR
numberOfPersons = 10
numberOfVideosPerPerson = 5
signs_list = ['nacer','comida','brillante', 'mujer', 'hijo', 'hombre', 'lejos', 'aprender', 'espumadera','amargo']
build_parquet('nacer-amargo')

signs_list = ['leche','Uruguay','pais','donde','ninguno','nombre','perfume','sordo','comprar','encontrar', 'nave espacial']
build_parquet('leche-nave_escpacial')

Armando parquet: $nacer-amargo


KeyboardInterrupt: 

In [52]:
# videos_dataframe[:'right_hand_p20_x']
print(len(videos_dataframe.axes[0]))
print(len(videos_dataframe.axes[1]))

100500
1087


In [8]:
dataframe1 = pd.read_parquet(os.path.join('./nacer-amargo'))
dataframe2 = pd.read_parquet(os.path.join('./leche-nave_espacial'))
concatenated_dataframes = pd.concat([dataframe1, dataframe2], axis=0, ignore_index=True)
concatenated_dataframes.to_parquet(os.path.join('parquet_final'))

In [9]:
print(len(concatenated_dataframes.axes[0]))
print(len(concatenated_dataframes.axes[1]))

211050
1087


In [69]:
concatenated_dataframes.iloc[[50249]]

,face_p0_x,face_p0_y,face_p1_x,face_p1_y,face_p2_x,face_p2_y,face_p3_x,face_p3_y,face_p4_x,face_p4_y,...,right_hand_p16_y,right_hand_p17_x,right_hand_p17_y,right_hand_p18_x,right_hand_p18_y,right_hand_p19_x,right_hand_p19_y,right_hand_p20_x,right_hand_p20_y,sign
50249,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,hijo
